In [2]:
import os
import operator
import pandas as pd
import re
import ast

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
from sklearn.metrics import mean_squared_error

import tensorflow as tf
import tensorflow_hub as hub

import numpy as np
import time
from datetime import datetime 

G:\AnacondaSetup\envs\New_Env\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
G:\AnacondaSetup\envs\New_Env\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
G:\AnacondaSetup\envs\New_Env\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
G:\AnacondaSetup\envs\New_Env\lib\site-packages\tensorflow\python\framework\dtypes.py:529: 

In [3]:
def embed_useT(module):
    with tf.Graph().as_default():
        sentences = tf.placeholder(tf.string)
        embed = hub.Module(module)
        embeddings = embed(sentences)
        session = tf.train.MonitoredSession()
    return lambda x: session.run(embeddings, {sentences: x})

In [4]:
google_universal_encoder_loading_time=0

google_universal_encoder_loading_start_time = datetime.now() 
embed_fn = embed_useT('G:\sentence_wise_email\module\module_useT') #loading the model
google_universal_encoder_loading_time =format(datetime.now() - google_universal_encoder_loading_start_time)

print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - google_universal_encoder_loading_start_time))

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Graph was finalized.


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


Time elapsed (hh:mm:ss.ms) 0:00:55.399727


In [5]:
def get_Topic(df,id):  
    return df['Topic_name'][id] # Just reads the results out of the dictionary.

def get_question(df,id):  
    return df['Question'][id] # Just reads the results out of the dictionary.

def get_Distractor(df,id):  
    return df['List_of_distractors'][id] # Just reads the results out of the dictionary.

def get_answer(df,id):  
    return df['Answer'][id] # Just reads the results out of the dictionary.

def get_Distractor_without_answer(df,id):  
    return df['dis_without_answer'][id] # Just reads the results out of the dictionary.

def load_data(csv_dir):
    #upload enhanced_data.csv
    df = pd.read_csv(csv_dir)
    
    counter=0
    
    for i in df['List_of_distractors']:
        df['List_of_distractors'][counter]=ast.literal_eval(i)
        counter=counter+1
    
    counter=0
    for i in df['dis_without_answer']:
        df['dis_without_answer'][counter]=ast.literal_eval(i)
        counter=counter+1
    return df

In [140]:
def recommend_most_similar_questions(user_input, csv_dir ):

    df = load_data(csv_dir)
    '''
    We will treat the user input as a document , as add it in the collection of skill documents we have.
    '''
    all_docs=[]
    for i in df['Topic_name']:
        all_docs.append(i)

    #all_docs.append(user_input)
    
    '''
    embedding our documenets 
    '''
    embedding_time=0
    embedding_start_time = datetime.now() 
    encoding_matrix = embed_fn(all_docs)
    #print(encoding_matrix.shape)
    embedding_time =format(datetime.now() - embedding_start_time)

    print('embedding_time (hh:mm:ss.ms) {}'.format(datetime.now() - embedding_start_time))
    
    
    embedding_user_time = 0
    embedding_user_start_time = datetime.now() 
    encoding_user_vector = embed_fn([user_input])
    #print(encoding_user_vector.shape)
    embedding_user_time =format(datetime.now() - embedding_user_start_time)

    print('embedding_user_time (hh:mm:ss.ms) {}'.format(datetime.now() - embedding_user_start_time))
    
    
    similarity_time=0
    similarity_start_time = datetime.now() 
    '''
    get the cosine similarity between documents 
    '''
    cosine_similarities = linear_kernel(encoding_user_vector, encoding_matrix)
    
    '''
    sort cosine similarity values to get the most high 100 scores
    '''

    related_docs_indices = cosine_similarities[0][:-2].argsort()[:-0:-1]

    related_docs_indices=related_docs_indices[0:31] #we get always the best 30 questions
    
    list_result=list()
    recommended_questions_list=list()
    all_questions=list()
    i=0
    for rec in cosine_similarities[-1][related_docs_indices]: 
        whole_question_list=list()
        anwer = get_answer(df,related_docs_indices[i])
        dis_without_ans = get_Distractor_without_answer(df,related_docs_indices[i])
        Question=get_question(df,related_docs_indices[i])

        whole_question_list.append(anwer)
        for each_element in dis_without_ans:
            whole_question_list.append(each_element)
        whole_question_list.append(Question)

        all_questions.append(whole_question_list)    
        i=i+1
    
    similarity_time =format(datetime.now() - similarity_start_time)
    print('similarity_time (hh:mm:ss.ms) {}'.format(datetime.now() - similarity_start_time))
    
    dic = {}
    c=0
    for each_list in all_questions:
        dic[c] = each_list
        c=c+1
        
    return dic

# try -----------------------------------------------------------------

In [141]:
user_input="Recursion (adjective: recursive) occurs when a thing is defined in terms of itself or of its type. Recursion is used in a variety of disciplines ranging from linguistics to logic. The most common application of recursion is in mathematics and computer science, where a function being defined is applied within its own definition. While this apparently defines an infinite number of instances (function values), it is often done in such a way that no infinite loop or infinite chain of references can occur. Recursion is the process a procedure goes through when one of the steps of the procedure involves invoking the procedure itself. A procedure that goes through recursion is said to be 'recursive'.[3] To understand recursion, one must recognize the distinction between a procedure and the running of a procedure. A procedure is a set of steps based on a set of rules, while the running of a procedure involves actually following the rules and performing the steps. Recursion is related to, but not the same as, a reference within the specification of a procedure to the execution of some other procedure. When a procedure is defined as such, this immediately creates the possibility of an endless loop; recursion can only be properly used in a definition if the step in question is skipped in certain cases so that the procedure can complete. But even if it is properly defined, a recursive procedure is not easy for humans to perform, as it requires distinguishing the new from the old, partially executed invocation of the procedure; this requires some administration as to how far various simultaneous instances of the procedures have progressed. For this reason, recursive definitions are very rare in everyday situations."

In [142]:
dic=recommend_most_similar_questions(user_input,'questions_recommendation_system_pl.csv')

G:\AnacondaSetup\envs\New_Env\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
G:\AnacondaSetup\envs\New_Env\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


embedding_time (hh:mm:ss.ms) 0:00:07.372718
embedding_user_time (hh:mm:ss.ms) 0:00:00.235380
similarity_time (hh:mm:ss.ms) 0:00:00.019947


In [143]:
dic

{0: ['Run time error',
  'Compile time error',
  'Logical error',
  'No error',
  'In the absence of a exit condition in a recursive function, the following error is given __________'],
 1: ['24',
  '4',
  '12',
  '10',
  'What will be the output of the following C code?\n#include<stdio.h>\nmain()\n{\nint n;\nn=f1(4);\nprintf("%d",n);\n}\nf1(int x)\n{\nint b;\nif(x==1)\nreturn 1;\nelse\nb=x*f1(x-1);\nreturn b;\n}'],
 2: ['Last in first out',
  'First in first out',
  'First in last out',
  'Last in last out',
  'The principle of stack is __________'],
 3: ['++++2',
  '+++++2',
  '+++++',
  '2',
  'What will be the output of the following C code?\n#include<stdio.h>\nmain()\n{\nint n,i;\nn=f(6);\nprintf("%d",n);\n}\nf(int x)\n{\nif(x==2)\nreturn 2;\nelse\n{\nprintf("+");\nf(x-1);\n}\n}'],
 4: ['Infinite number of times',
  '9 times',
  '10 times',
  '0 times',
  'How many times is ‘a’ printed when the following C code is executed?\n#include<stdio.h>\nmain()\n{\nint a;\na=f1(10);\nprintf(

In [144]:
user_input="A neural networks is a network or circuit of neurons, or in a modern sense, an artificial neural network, composed of artificial neurons or nodes.[1] Thus a neural network is either a biological neural network, made up of real biological neurons, or an artificial neural network, for artificial intelligence . The connections of the biological neuron are modeled as weights. A positive weight reflects an excitatory connection, while negative values mean inhibitory connections. All inputs are modified by a weight and summed. This activity is referred to as a linear combination. Finally, an activation function controls the amplitude of the output. For example, an acceptable range of output is usually between 0 and 1, or it could be −1 and 1. These artificial networks may be used for predictive modeling, adaptive control and applications where they can be trained via a dataset. Self-learning resulting from experience can occur within networks, which can derive conclusions from a complex and seemingly unrelated set of information.[2]"

In [145]:
dic=recommend_most_similar_questions(user_input,'questions_recommendation_system_ai.csv')

G:\AnacondaSetup\envs\New_Env\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
G:\AnacondaSetup\envs\New_Env\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


embedding_time (hh:mm:ss.ms) 0:00:03.768885
embedding_user_time (hh:mm:ss.ms) 0:00:00.255317
similarity_time (hh:mm:ss.ms) 0:00:00.003989


In [146]:
dic

{0: ['a single layer feed-forward neural network with pre-processing',
  'an auto-associative neural network',
  'a double layer auto-associative neural network',
  'a neural network that contains feedback',
  'What is perceptron?'],
 1: ['True – perceptrons can do this but are unable to learn to do it – they have to be explicitly hand-coded',
  'True – this works always, and these multiple perceptrons learn to classify even complex problems',
  'False – perceptrons are mathematically incapable of solving linearly inseparable functions, no matter what you do',
  'False – just having a single perceptron is enough',
  'Having multiple perceptrons can actually solve the XOR problem satisfactorily: this is because each perceptron can partition off a linear part of the space itself, and they can then combine their results.'],
 2: ['Because they are the only class of problem that Perceptron can solve successfully',
  'Because they are the only class of problem that network can solve successf

In [147]:
user_input="In software engineering, a software design pattern is a general, reusable solution to a commonly occurring problem within a given context in software design. It is not a finished design that can be transformed directly into source or machine code. Rather, it is a description or template for how to solve a problem that can be used in many different situations. Design patterns are formalized best practices that the programmer can use to solve common problems when designing an application or system. Object-oriented design patterns typically show relationships and interactions between classes or objects, without specifying the final application classes or objects that are involved. Patterns that imply mutable state may be unsuited for functional programming languages, some patterns can be rendered unnecessary in languages that have built-in support for solving the problem they are trying to solve, and object-oriented patterns are not necessarily suitable for non-object-oriented languages. Design patterns may be viewed as a structured approach to computer programming intermediate between the levels of a programming paradigm and a concrete algorithm."

In [148]:
dic=recommend_most_similar_questions(user_input,'questions_recommendation_system_sw.csv')

G:\AnacondaSetup\envs\New_Env\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
G:\AnacondaSetup\envs\New_Env\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


embedding_time (hh:mm:ss.ms) 0:00:06.510445
embedding_user_time (hh:mm:ss.ms) 0:00:00.129653
similarity_time (hh:mm:ss.ms) 0:00:00.004986


In [149]:
dic

{0: ['Player-Role Pattern',
  'Abstraction-Occurrence Pattern',
  'General Hierarchy Pattern',
  'Singleton Pattern',
  'You want to avoid multiple inheritance. Which design pattern would you choose?'],
 1: ['all of the mentioned',
  'encapsulating the knowledge of which document subclass to is to be created and',
  'moving this knowledge out of the framework',
  'instantiating the application specific documents without knowing their class',
  'In factory method pattern, the framework must instantiate classes but it only knows about the abstract classes, which it cannot initiate. How would one solve this problem?'],
 2: ['False',
  'True',
  'Design patterns does not follow the concept of software reuse.'],
 3: ['All of the mentioned',
  'Component-based software engineering',
  'Reusability in general',
  'None of the mentioned',
  'The use of design patterns for the development of object-oriented software has important implications for'],
 4: ['All of the mentioned',
  'Behavioral',
